## Load necessary libraries

In [1]:
# import re
import pandas as pd
from gensim import corpora, models
import pandas as pd
import gensim
# import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../utils")

from Helper_Function_MachineLearning_Clustering import *

%aimport Helper_Function_MachineLearning_Clustering

## Load Data and split into feature and target

In [3]:
posts_df = pd.read_pickle('../data/interum/text_target.pkl')
# convert into features and target 
feature = posts_df['tokens']
label = posts_df['target']  # label 

In [4]:
# remove words longer than length 2
feature = feature.apply(
    lambda x: [w for w in x if len(w) > 2])

## feature engineering for clustering

In [5]:
# dictionary for train 
dictionary = gensim.corpora.Dictionary(feature)
dictionary.filter_extremes(no_below =1, no_above=0.05, keep_n=50000)

In [6]:
# create bag of words 
bow = [dictionary.doc2bow(doc) for doc in feature]
# tfidf for bow 
tfidf = models.TfidfModel(bow)
corpus_tfidf = tfidf[bow]

In [7]:
# generate a model for bag of words with 5 topics 
lda_model = gensim.models.LdaMulticore(
    bow, num_topics=5, id2word=dictionary, passes=2, workers=4,random_state=42)
# generate a model for bag of tfidf with 5 topics 
lda_model_tfidf = gensim.models.LdaMulticore(
    corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=4,random_state=42)

In [8]:
topic_top_word(lda_model)

,topic_0,topic_1,topic_2,topic_3,topic_4
0,byte,android,std,org,log
1,long,script,foo,key,source
2,thread,item,event,self,org
3,char,model,option,import,framework
4,size,date,template,lib,module


In [9]:
topic_top_word(lda_model_tfidf)

,topic_0,topic_1,topic_2,topic_3,topic_4
0,script,date,std,foo,thread
1,byte,item,foo,key,div
2,event,script,vector,print,arraylist
3,image,android,date,log,private
4,long,form,template,bar,log


## Predict use both models 

In [10]:
# predict with bow lda
pred_bow = []
for i in bow:
    result = lda_model[i]
    if len(result) > 1:
        pred_bow.append(sorted(result, key=lambda x: x[1], reverse = True )[0][0])
    else:
        pred_bow.append(result[0][0])    

In [11]:
# predict with tfidf lda
pred_tdif = []
for i in bow:
    result = lda_model_tfidf[i]
    if len(result) > 1:
        pred_tdif.append(sorted(result, key=lambda x: x[1], reverse = True )[0][0])
    else:
        pred_tdif.append(result[0][0])
        

In [12]:
# combine results 
result = pd.DataFrame([pred_bow,pred_tdif,label]).T
result.columns = ['bow','tdif','true_label']

In [13]:
# combine model results with original text to see content 
combined_df = pd.concat([posts_df[['text']],result], axis = 1)

## Bow and topic 0 to 4 we will check 5 random texts

In [14]:
for topic in range(0,5,1):
    i = 0
    print(f'for model bow and topic {topic}:')
    print('------------------------------')
    while i < 5:
        print(text_topic(combined_df,'bow',topic))   
        print('------------------------------')
        i += 1

for model bow and topic 0:
------------------------------


How do I read every line of a file in Python and store each line as an element
in an array?

I want to read the file line by line and each line is appended to the end of
the array.


------------------------------


I want to find the best approach for converting `bool[]` into `object[]` in C#
.NET 4.0.  
Now I have this variables:

    
    
    object[] objectArray = new object [] { true false true }; string[] stringArray = new string[] { true  false  true }; bool[] boolArray = new bool[] { true false true }; 

All are created fine. For 'clear types' suc as `bool` and `object` boxing
works fine (`object o = true;`). But in this case I can do conversion only
from a string array to an object array but not from a boolean array:

    
    
    objectArray = stringArray; // OK objectArray = boolArray; // WRONG Cannot implicitly convert bool[] to object[] 

Also in some methods I am sending a list of object arrays. As in previous 



> **Possible Duplicate:**  
>  Exception in thread "main" java.lang.UnsupportedClassVersionError: a
(Unsupporte d major.minor version 51.0)

i have developed an web application in java using jdk 1.7.0 Eclipse IDE
(Indigo) and is running that application on the tomcat(apache-tomcat-7.0.23)
configured in eclipse ide.

When i tried to run my application through IDE it runs fine but when i created
its war and placed it in apache deployment folder(webapps) and run it from
outside the IDE the start page gets successfully loaded but when i tried to do
any operation over it it gives me an error

    
    
    UnsupportedClassVersionError unsupported major.minor version 51.0 unable to load class bean.myclassname 

i have checked the java version outside ide its jdk 1.7.0 and also the
JAVA_HOME environment variable is set to C:\Program Files\Java\jdk1.7.0_01 .


------------------------------


I'm trying to get matplotlib up and running on OS X 10.8.4. I've installed
matplotlib and the depend

## tfidf and topic 0 to 4 we will check 5 random texts¶

In [15]:
for topic in range(0,5,1):
    i = 0
    print(f'for model bow and topic {topic}:')
    print('------------------------------')
    while i < 5:
        print(text_topic(combined_df,'tdif',topic))   
        print('------------------------------')
        i += 1

for model bow and topic 0:
------------------------------


I'd like to create a file with path `x` using python. I've been using
`os.system(y)` where `y = 'touch %s' % (x)`. I've looked for a non-directory
version of `os.mkdir` but I haven't been able to find anything. Is there a
tool like this to create a file without opening it or using system or
popen/subprocess?


------------------------------


I'd like to have a method that transforms the first character of a string into
lower case.

My approaches:

1.

    
    
    public static string ReplaceFirstCharacterToLowerVariant(string name) { return String.Format( {0}{1}  name.First().ToString().ToLowerInvariant() name.Substring(1)); } 

2.

    
    
    public static IEnumerable<char> FirstLetterToLowerCase(string value) { var firstChar = (byte)value.First(); return string.Format( {0}{1}  (char)(firstChar + 32) value.Substring(1)); } 

What would be your approach?


------------------------------


So once again I make a nice pyth



Is there any utility for deep cloning for java collections:

  * Arrays
  * Lists
  * Maps

NOTE: prefer some solution without usage of serialization but with use of
Object.clone() method. I can be sure that my custom object will implement
clone() method and will use only java-standard classes that are cloneable...


------------------------------
for model bow and topic 4:
------------------------------


So I have this directive called say `mySave` it's pretty much just this

    
    
    app.directive('mySave' function($http) { return function(scope element attrs) { element.bind( click  function() { $http.post('/save' scope.data).success(returnedData) { // callback defined on my utils service here // user defined callback here from my-save-callback perhaps? } }); } }); 

the element itself looks like this

    
    
    <button my-save my-save-callback= callbackFunctionInController() >save</button> 

callbackFunctionInController is for now just

    
    
    $scope.callbackFunct